In [11]:
# import
import pandas as pd
import os
import json
import random

In [12]:
path_groundTruth_excel = '../groundTruth/groundTruth.xlsx'
df_groundTruth = pd.read_excel(path_groundTruth_excel, engine='openpyxl')

list_groundTruth_address = df_groundTruth['address'].tolist()
print('Number of groundTruth addresses: ', len(list_groundTruth_address))

Number of groundTruth addresses:  69


In [13]:
# 指定文件夹路径
path_hex_folder = '../groundTruth/hex'
path_sol_folder = '../groundTruth/sol'

# 获取文件夹中的所有文件名，不包括后缀
list_filenames_hex = [os.path.splitext(f)[0] for f in os.listdir(path_hex_folder) if os.path.isfile(os.path.join(path_hex_folder, f))]
list_filenames_sol = [os.path.splitext(f)[0] for f in os.listdir(path_sol_folder) if os.path.isfile(os.path.join(path_sol_folder, f))]

In [14]:
# 找到还缺少的hex和sol文件
def GetLackAddress(_list_groundTruth_address, _list_filenames_hex, _list_filenames_sol):
    _list_hex_lack = []
    _list_sol_lack = []
    for _address in _list_groundTruth_address:
        if _address not in _list_filenames_hex:
            _list_hex_lack.append(_address)
        if _address not in _list_filenames_sol:
            _list_sol_lack.append(_address)
    return _list_hex_lack, _list_sol_lack

In [5]:
list_hex_lack, list_sol_lack = GetLackAddress(list_groundTruth_address, list_filenames_hex, list_filenames_sol)
print('The number of lack addresses: hex: ', len(list_hex_lack), '; sol: ', len(list_sol_lack))

The number of lack addresses: hex:  0 ; sol:  0


分析实验结果

In [15]:
total_set_groundTruth = set()
set_truth_one = set()
set_truth_two = set()
set_truth_three = set()

df_Mint = pd.DataFrame(columns=['Address', 'Label', 'Result'])
df_Mint['Address'] = df_groundTruth['address']
df_Mint['Label'] = df_groundTruth['Mint']

df_Leak = pd.DataFrame(columns=['Address', 'Label', 'Result'])
df_Leak['Address'] = df_groundTruth['address']
df_Leak['Label'] = df_groundTruth['Leak']

df_Limit = pd.DataFrame(columns=['Address', 'Label', 'Result'])
df_Limit['Address'] = df_groundTruth['address']
df_Limit['Label'] = df_groundTruth['Limit']

path_result_json = '/pro/linzw/CRPWarner/zeroruger/gigahorse-toolchain/groundtruth/0123_2247/results.json'
with open(path_result_json, 'r') as f:
    list_result = json.load(f)

for item in list_result:
    _count = 0
    address = item[0][:item[0].find('.hex')]
    row = df_Mint.loc[df_Mint['Address'] == address].index[0]

    list_result_output = item[1]
    if 'Mint_0106' in list_result_output:
        df_Mint.at[row, 'Result'] = 1
        total_set_groundTruth.add(address)
        _count += 1
    else:
        df_Mint.at[row, 'Result'] = 0

    if 'Leak_0104' in list_result_output:
        df_Leak.at[row, 'Result'] = 1
        total_set_groundTruth.add(address)
        _count += 1
    else:
        df_Leak.at[row, 'Result'] = 0

    if 'Limit_0102' in list_result_output:
        df_Limit.at[row, 'Result'] = 1
        total_set_groundTruth.add(address)
        _count += 1
    else:
        df_Limit.at[row, 'Result'] = 0

    if _count == 1:
        set_truth_one.add(address)
    elif _count == 2:
        set_truth_two.add(address)
    elif _count == 3:
        set_truth_three.add(address)

print('Number of all contracts: 69')
print('Number of one risk: ', len(set_truth_one))
print('Number of two risk: ', len(set_truth_two))
print('Number of three risk: ', len(set_truth_three))
print('Number of contracts with risk: ', len(total_set_groundTruth))
print('Number of contracts without risk: ', 69 - len(total_set_groundTruth))

Number of all contracts: 69
Number of one risk:  22
Number of two risk:  17
Number of three risk:  0
Number of contracts with risk:  39
Number of contracts without risk:  30


In [7]:
# 分析Mint的实验结果
df_true_positives_mint = df_Mint[(df_Mint['Label'] == 1) & (df_Mint['Result'] == 1)]
df_false_positives_mint = df_Mint[(df_Mint['Label'] == 0) & (df_Mint['Result'] == 1)]
df_true_negatives_mint = df_Mint[(df_Mint['Label'] == 0) & (df_Mint['Result'] == 0)]
df_false_negatives_mint = df_Mint[(df_Mint['Label'] == 1) & (df_Mint['Result'] == 0)]

num_tp_mint = df_true_positives_mint.shape[0]
num_fp_mint = df_false_positives_mint.shape[0]
num_tn_mint = df_true_negatives_mint.shape[0]
num_fn_mint = df_false_negatives_mint.shape[0]

precision_mint = num_tp_mint / (num_tp_mint + num_fp_mint)
recall_mint = num_tp_mint / (num_tp_mint + num_fn_mint)
if precision_mint + recall_mint == 0:
    f1_mint = 0
else:
    f1_mint = 2 * (precision_mint * recall_mint) / (precision_mint + recall_mint)

print('The result of Mint: ')
print('Precision:', precision_mint, '; Recall:', recall_mint, '; F1-score:', f1_mint)
print('The number of fp:', num_fp_mint, '; fn:', num_fn_mint, '; tp:', num_tp_mint, '; tn:', num_tn_mint)

The result of Mint: 
Precision: 0.9473684210526315 ; Recall: 0.9 ; F1-score: 0.9230769230769231
The number of fp: 1 ; fn: 2 ; tp: 18 ; tn: 48


In [8]:
# 分析Limit的实验结果
df_true_positives_limit = df_Limit[(df_Limit['Label'] == 1) & (df_Limit['Result'] == 1)]
df_false_positives_limit = df_Limit[(df_Limit['Label'] == 0) & (df_Limit['Result'] == 1)]
df_true_negatives_limit = df_Limit[(df_Limit['Label'] == 0) & (df_Limit['Result'] == 0)]
df_false_negatives_limit = df_Limit[(df_Limit['Label'] == 1) & (df_Limit['Result'] == 0)]

num_tp_limit = df_true_positives_limit.shape[0]
num_fp_limit = df_false_positives_limit.shape[0]
num_tn_limit = df_true_negatives_limit.shape[0]
num_fn_limit = df_false_negatives_limit.shape[0]

precision_limit = num_tp_limit / (num_tp_limit + num_fp_limit)
recall_limit = num_tp_limit / (num_tp_limit + num_fn_limit)
f1_limit = 2 * (precision_limit * recall_limit) / (precision_limit + recall_limit)

print('The result of Limit: ')
print('Precision:', precision_limit, '; Recall:', recall_limit, '; F1-score:', f1_limit)
print('The number of fp:', num_fp_limit, '; fn:', num_fn_limit, '; tp: ', num_tp_limit, '; tn: ', num_tn_limit)

The result of Limit: 
Precision: 0.9310344827586207 ; Recall: 0.9 ; F1-score: 0.9152542372881356
The number of fp: 2 ; fn: 3 ; tp:  27 ; tn:  37


In [16]:
# 分析Leak的实验结果
df_true_positives_leak = df_Leak[(df_Leak['Label'] == 1) & (df_Leak['Result'] == 1)]
df_false_positives_leak = df_Leak[(df_Leak['Label'] == 0) & (df_Leak['Result'] == 1)]
df_true_negatives_leak = df_Leak[(df_Leak['Label'] == 0) & (df_Leak['Result'] == 0)]
df_false_negatives_leak = df_Leak[(df_Leak['Label'] == 1) & (df_Leak['Result'] == 0)]

num_tp_leak = df_true_positives_leak.shape[0]
num_fp_leak = df_false_positives_leak.shape[0]
num_tn_leak = df_true_negatives_leak.shape[0]
num_fn_leak = df_false_negatives_leak.shape[0]

precision_leak = num_tp_leak / (num_tp_leak + num_fp_leak)
recall_leak = num_tp_leak / (num_tp_leak + num_fn_leak)
if precision_leak + recall_leak == 0:
    f1_leak = 0
else:   
    f1_leak = 2 * (precision_leak * recall_leak) / (precision_leak + recall_leak)

print('The result of Leak: ')
print('Precision:', precision_leak, '; Recall:', recall_leak, '; F1-score:', f1_leak)
print('The number of fp:', num_fp_leak, '; fn:', num_fn_leak, '; tp:', num_tp_leak, '; tn:', num_tn_leak)

The result of Leak: 
Precision: 0.875 ; Recall: 0.7777777777777778 ; F1-score: 0.823529411764706
The number of fp: 1 ; fn: 2 ; tp: 7 ; tn: 59


分别分析GroundTruth的实验结果

In [9]:
df_false_positives_leak

,Address,Label,Result
5,0x11CBC781DadAAD13fc3a361772C80B1C027820AF,0,1


In [10]:
df_false_negatives_leak

,Address,Label,Result
31,0x9dB8a10C7FE60d84397860b3aF2E686D4F90C2b7,1,0
34,0x42269AC712372AC89A158ad5a32806c6b6782d66,1,0
51,0xa942890d7FC60F0D4a516f63dd273DcDE72aE6c9,1,0


分析大规模实验的实验结果

In [13]:
dict_large = {}
set_large_mint = set()
set_large_leak = set()
set_large_limit = set()
set_large_one = set()
set_large_two = set()
set_large_three = set()

path_large_json = '/pro/linzw/CRPWarner/zeroruger/gigahorse-toolchain/large/0123_2308/results.json'
with open(path_large_json, 'r') as f:
    list_large_result = json.load(f)

for item in list_large_result:
    _count = 0
    address = item[0][:item[0].find('.hex')]
    dict_pattern = {}

    list_result_output = item[1]
    if 'Mint_0106' in list_result_output:
        dict_pattern['Mint'] = True
        set_large_mint.add(address)
        _count += 1
    else:
        dict_pattern['Mint'] = False

    if 'Leak_0104' in list_result_output:
        dict_pattern['Leak'] = True
        set_large_leak.add(address)
        _count += 1
    else:
        dict_pattern['Leak'] = False

    if 'Limit_0102' in list_result_output:
        dict_pattern['Limit'] = True
        set_large_limit.add(address)
        _count += 1
    else:
        dict_pattern['Limit'] = False
    
    dict_large[address] = dict_pattern

    if _count == 1:
        set_large_one.add(address)
    elif _count == 2:
        set_large_two.add(address)
    elif _count == 3:
        set_large_three.add(address)

total_set = set()
total_set = set_large_mint.union(set_large_leak, set_large_limit)

print('Number of all contracts: ', len(dict_large))
print('Number of one risk: ', len(set_large_one))
print('Number of two risk: ', len(set_large_two))
print('Number of three risk: ', len(set_large_three))
print('Number of contracts with risk: ', len(total_set))
print('Number of contracts without risk: ', len(dict_large) - len(total_set))


Number of all contracts:  13483
Number of one risk:  2181
Number of two risk:  1416
Number of three risk:  571
Number of contracts with risk:  4168
Number of contracts without risk:  9315


In [3]:
print('The number of mint: ', len(set_large_mint))
sampled_mint = random.sample(set_large_mint, 72)
print('Sampled Address: ')
count = 1
for address in sampled_mint:
    print(count, ': ', address)
    count += 1


The number of mint:  2775
Sampled Address: 
1 :  0x645b6d66a07479bca259aaaaf0066d415e1d4b44
2 :  0x8d46facfb889740a345755e624942257dc87b5ac
3 :  0x7d227fe3c5885c3875180e03c548d09ebe1b0119
4 :  0xb363a3c584b1f379c79fbf09df015da5529d4dac
5 :  0xaba501a8ca62451f4af9e88b9f7f534ceec84c2a
6 :  0x616f9cb89319194935543f445266735bfce524d6
7 :  0x2ea20ba1c704db0822b78e146cd7a5f3b0342fb1
8 :  0x55b65bdbe29a1b2d29e28038b09075bac320d11d
9 :  0x7dd2f15b4a8c8e14da5eaa7b635221d4b39c41cc
10 :  0x9157c8359a1e349e599665bf83e22d32c69b9daf
11 :  0x08769a9b479a4b20e796194d960cc407fc66359a
12 :  0xf18ef338df7770abf0d20d09dff86c7398a91046
13 :  0x2a90a1536dbadcb3be3c3e8af0019478c449fa5c
14 :  0x8c01ada8e708993a891d57d1b3169479a20acb3a
15 :  0x89cf87c35e69a9b84f7a3e50eaf54bfc3cabc377
16 :  0x2f4f6c055a7c8ab529f5b01f6616164aaeec18ad
17 :  0x1cf33ead073351b3fc63c1f4f35dc10791fe6c7c
18 :  0x0c29d192dd5443fb5f00e5fe52f61d793025643b
19 :  0x7fccf800568747b178c6cbbe4bf3d147df75ac61
20 :  0x54ee540b38fc6e0548172fe651

In [4]:
print('The number of limit: ', len(set_large_limit))
sampled_limit = random.sample(set_large_limit, 20)
print('Sampled Address: ')
count = 1
for address in sampled_limit:
    print(count, ': ', address)
    count += 1

The number of limit:  2796
Sampled Address: 
1 :  0x76bc9e61a1904b82cbf70d1fd9c0f8a120483bbb
2 :  0x7f3e7af72b565d508ce63abbfdefd0161390c9ed
3 :  0x897dc0cca9bb43f7601e477e721ef4dea4453f07
4 :  0xb1733c9c3cb5d98c2c35db27e30de26081d4d30a
5 :  0xe79b55a739abd64c62e8a642afbf02582c008c88
6 :  0x4c567c3363cc42c5a42c6d8bf01503fd1d0b91cd
7 :  0x40165bd6d657b63cb6f8f4a3525ac05bbf74bd93
8 :  0x76a6b003c39c1eb38b96aaaa1f551e4c6ad3de5c
9 :  0x9ab4fd2234d75cfa894c394b367e48055b531e56
10 :  0x02d3aea48b443a0026ed9cbc91b97d7335aba323
11 :  0x3f1db2c3e98232971ec8ccfea8be865228763fef
12 :  0xc5284e12632a01046d01a8dc886466f82071254b
13 :  0x5ee5e63f3225e5b070be0fac1cf6c2ae2ade24d4
14 :  0x9c0a1bbf01aee832afafeac003a57ff508a2a386
15 :  0xdf859c9878ef5e742d7bbe3c22a496c088c89fa9
16 :  0x8f3a9953c5ca4449d15721cd6e11ebc15a89e569
17 :  0x2b36968bf3c77253266aa382562fa3fdd7fa1667
18 :  0xe2c43d2c6d6875c8f24855054d77b5664c7e810f
19 :  0xbed451b9d7c1e0458a2e57888a001c20e3b65f44
20 :  0x56ba1d5ba4d1b6efc3993493c

In [12]:
print('The number of leak: ', len(set_large_leak))
sampled_leak = random.sample(set_large_leak, 1)
print('Sampled Address: ')
count = 1
for address in sampled_leak:
    print(count, ': ', address)
    count += 1

The number of leak:  1155
Sampled Address: 
1 :  0x46043c9093ff4d0a796bd1aee7e80254420368dd
